# **Back Order Prediction Final Model for Deployment**

In [29]:
#import libraries
import pandas as pd
import numpy as np

In [30]:
data=pd.read_csv("Training_Dataset_v2.csv")                         

I m considering only those features to train our model which are selected by our feature selection steps by random  forest features_importance.

In [31]:
train_df=data[['national_inv', 'forecast_3_month', 'forecast_6_month',
       'forecast_9_month', 'sales_1_month', 'sales_3_month', 'sales_6_month',
       'sales_9_month', 'min_bank', 'perf_6_month_avg', 'perf_12_month_avg','went_on_backorder']]
train_df.head()

,national_inv,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,sales_9_month,min_bank,perf_6_month_avg,perf_12_month_avg,went_on_backorder
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-99.00,-99.00,No
1,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.99,0.99,No
2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-99.00,-99.00,No
3,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.10,0.13,No
4,8.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2.0,-99.00,-99.00,No


In [32]:
#impute missing values
train_df.dropna(how='any',inplace=True)

In [33]:
Categorical_features=train_df.select_dtypes(include=['object'])           #encoding of categorical features
for col in Categorical_features:
    train_df[col] = train_df[col].map({"Yes" : 0, "No" : 1})

In [34]:
X = train_df.drop(["went_on_backorder"],axis=1)
y=train_df['went_on_backorder']

In [35]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train,y_test = train_test_split(X,y,test_size = 0.3)

In [36]:
print("Train Set Size : ",x_train.shape)
print("Train Target Set Size : ",y_train.shape)
print("Test  Set Size : ",x_test.shape)
print("Test  Target Set Size : ",y_test.shape)

Train Set Size :  (1181502, 11)
Train Target Set Size :  (1181502,)
Test  Set Size :  (506358, 11)
Test  Target Set Size :  (506358,)


In [37]:
#ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [38]:
from imblearn.over_sampling import SMOTE    #lets balance the data
sm = SMOTE(random_state=12, ratio = 1.0)
x_train, y_train = sm.fit_sample(x_train, y_train)
x_train.shape, y_train.shape

((2347254, 11), (2347254,))

# **Model Building**

In [39]:
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score,classification_report
from sklearn.ensemble import RandomForestClassifier

**Random Forest**

In [40]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(x_train, y_train)
Y_prediction = random_forest.predict(x_test)

random_forest.score(x_train, y_train)
acc_random_forest = round(random_forest.score(x_train, y_train) * 100, 2)
print(acc_random_forest)
print(random_forest.score(x_test,y_test))

99.61
0.9904968421551551


In [41]:
print(classification_report(y_test,Y_prediction))
# Confusion Matrix
conf_mat = confusion_matrix(y_test,Y_prediction)
print(conf_mat)

              precision    recall  f1-score   support

           0       0.34      0.42      0.37      3418
           1       1.00      0.99      1.00    502940

    accuracy                           0.99    506358
   macro avg       0.67      0.71      0.68    506358
weighted avg       0.99      0.99      0.99    506358

[[  1426   1992]
 [  2820 500120]]


**save Model**

In [42]:
import pickle

In [43]:
# saving the model to the local file system
filename = 'backorder_model.pickle'
pickle.dump(random_forest, open(filename, 'wb'))

In [44]:
# prediction using the saved model
loaded_model = pickle.load(open(filename, 'rb'))
a=loaded_model.predict(([[5,6,8,3,2,9,12,34,8,4,75]]))
a

array([1])